<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 4



### 6.1 Softmax Prediction
* To classify more than two classes



#### 1D



Three classes:
* `y = 0`
* `y = 1`
* `y = 2`



In [ ]:
import torch


x = torch.linspace(-2.0, 2.0, 41)
y = (x > -1)*1.0 + (x > 1)*1.0



In [ ]:
import matplotlib.pyplot as plt


plt.plot(x.numpy(), y.numpy(), '.')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)



#### 2D



### 6.2 Softmax Function



### 6.3 Softmax PyTorch



## Shallow Neural Networks



### 7.1 Neural Networks in One Dimension



### 7.2 Neural Networks More Hidden Neurons



### 7.3 Neural Networks with Multiple Dimensional Input



### 7.4 Multi-Class Neural Networks



### 7.5 Backpropagation



### 7.6 Activation Function

